In [1]:
import sys
import time
import random
import pygame

from pygame.surfarray import array3d

BLACK = pygame.Color(0, 0, 0)
WHITE = pygame.Color(255, 255, 255)
RED = pygame.Color(255, 0, 0)
GREEN = pygame.Color(0, 255, 0)

pygame 2.1.0 (SDL 2.0.16, Python 3.7.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class SnakeEnv:
    
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.window = pygame.display.set_mode((width, height))
        self.reset()
        
    def reset(self):
        self.window.fill(BLACK)
        self.snake_pos = [100, 50]
        self.snake_body = [[100, 50], [90, 50], [80, 50]]
        self.food_pos = self.spawn_food()
        self.food_spawn = True
        
        self.direction = "RIGHT"
        self.action = self.direction
        
        self.score = 0
        self.steps = 0
        print("GAME RESET")
        
    def change_direction(self, action, direction):
        if action == "UP" and direction != "DOWN":
            direction = "UP"
        if action == "DOWN" and direction != "UP":
            direction = "DOWN"
        if action == "LEFT" and direction != "RIGHT":
            direction = "LEFT"
        if action == "RIGHT" and direction != "LEFT":
            direction = "RIGHT"
        return direction
    
    def move(self, direction, snake_pos):
        if direction == "UP":
            snake_pos[1] -= 10
        elif direction == "DOWN":
            snake_pos[1] += 10
        elif direction == "LEFT":
            snake_pos[0] -= 10
        elif direction == "RIGHT":
            snake_pos[0] += 10
        return snake_pos
        
    def spawn_food(self):
        x = random.randrange(1, self.width//10)*10
        y = random.randrange(1, self.height//10)*10
        food = [x, y]
        if food in self.snake_body:
            return self.spawn_food()
        return food
    
    def eat(self):
        return self.snake_pos[0] == self.food_pos[0] and self.snake_pos[1] == self.food_pos[1]
    
    def human_step(self, event):
        action = None
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_UP:
                action = "UP"
            elif event.key == pygame.K_DOWN:
                action = "DOWN"
            elif event.key == pygame.K_LEFT:
                action = "LEFT"
            elif event.key == pygame.K_RIGHT:
                action = "RIGHT"
            elif event.key == pygame.K_ESCAPE:
                pygame.event.post(pygame.event.Event(pygame.QUIT))

        return action
    
    def display_score(self, color, font, size):
        score_font = pygame.font.SysFont(font, size)
        score_surface = score_font.render(f"Score: {self.score}", True, color)
        score_rect = score_surface.get_rect()
        score_rect.midtop = (self.width/10, 15)
        self.window.blit(score_surface, score_rect)
        
    def game_over(self):
        # Touch box
        if self.snake_pos[0] < 0 or self.snake_pos[0] > self.width-10:
            self.end_game()
        elif self.snake_pos[1] < 0 or self.snake_pos[1] > self.height-10:
            self.end_game()
            
        # Touch own body
        for block in self.snake_body[1:]:
            if self.snake_pos[0] == block[0] and self.snake_pos[1] == block[1]:
                self.end_game()
    
    def end_game(self):
        message = pygame.font.SysFont("arial", 45)
        message_surface = message.render("GAME HAS ENDED", True, RED)
        message_rect = message_surface.get_rect()
        message_rect.midtop = (self.width/2, self.height/4)
        self.window.fill(BLACK)
        self.window.blit(message_surface, message_rect)
        self.display_score(RED, "times", 20)
        pygame.display.flip()
        time.sleep(3)
        pygame.quit()
        sys.exit()

In [3]:
snake_env = SnakeEnv(600, 600)
difficulty = 10
fps_controller = pygame.time.Clock()
check_errors = pygame.init()

pygame.display.set_caption("Snake Game")

while True:
    # Human input
    for event in pygame.event.get():
        snake_env.action = snake_env.human_step(event)
    
    # Check direction
    snake_env.direction = snake_env.change_direction(snake_env.action, snake_env.direction)
    snake_env.snake_pos = snake_env.move(snake_env.direction, snake_env.snake_pos)
    
    # Check if we ate food
    snake_env.snake_body.insert(0, list(snake_env.snake_pos))
    if snake_env.eat():
        snake_env.score += 1
        snake_env.food_spawn = False
    else:
        snake_env.snake_body.pop()
    
    # Check if spawn new food
    if not snake_env.food_spawn:
        snake_env.food_pos = snake_env.spawn_food()
    snake_env.food_spawn = True
    
    # Drawing the snake
    snake_env.window.fill(BLACK)
    for pos in snake_env.snake_body:
        pygame.draw.rect(snake_env.window, GREEN, pygame.Rect(pos[0], pos[1], 10, 10))
    
    # Drawing of food
    pygame.draw.rect(snake_env.window, WHITE, pygame.Rect(snake_env.food_pos[0], snake_env.food_pos[1], 10, 10))
    
    # Check if end game
    snake_env.game_over()
    
    # Refresh game screen
    snake_env.display_score(WHITE, "consolas", 20)
    
    pygame.display.update()
    fps_controller.tick(difficulty)
    img = array3d(snake_env.window)

GAME RESET


SystemExit: 

D:\Programas\miniconda3\envs\myenv\lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
